# Setup

In [1]:
import sys
sys.path.insert(
    0, '/home/student.unimelb.edu.au/shorte1/Documents/TINT')

In [ ]:
import sys
sys.path.insert(0, '/home/563/esh563/TINT')

In [2]:
# Core
import datetime
import os
import glob
import tempfile
import pickle
import copy

# Analysis
import numpy as np
import pandas as pd
import xarray as xr

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Debugging
import pdb, traceback
# %load_ext line_profiler

# Radar Tools
import pyart
import tint
# import tint.animate
# from tint.visualisation import animate

import CPOL_func as cf
import caine_func as caf
import analysis

# Convenience functions
%load_ext autoreload
%autoreload 1
%aimport CPOL_func
%aimport caine_func
%aimport tint.visualisation.figures
%aimport tint.visualisation.animate
%aimport tint.visualisation.horizontal_helpers
%aimport tint.visualisation.vertical_helpers
%aimport tint.tracks
%aimport tint.objects
%aimport tint.grid_utils
%aimport tint.steiner
%aimport tint.matching
%aimport tint.rain
%aimport tint.cells
%aimport tint.tracks_helpers
%aimport tint.process_ERA5
%aimport tint.process_WRF

%pdb


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/student.unimelb.edu.au/shorte1/anaconda3/envs/CPOL_analysis/lib/python3.8/site-packages/networkx/readwrite/graphml.py:346: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.int, "int"), (np.int8, "int"),
/home/student.unimelb.edu.au/shorte1/anaconda3/envs/CPOL_analysis/lib/python3.8/site-packages/networkx/readwrite/gexf.py:220: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to us

Automatic pdb calling has been turned ON


In [3]:
base_dir = '/media/shorte1/Ewan\'s Hard Drive/phd/data/CPOL/'
save_dir = '/home/student.unimelb.edu.au/shorte1/Documents/TINT_tracks/'
fig_dir = '/home/student.unimelb.edu.au/shorte1/Documents/TINT_figures/'
ERA5_dir = '/media/shorte1/Ewan\'s Hard Drive/phd/data/era5/'
ERA5_dir += 'pressure-levels/reanalysis/'
WRF_dir = '/media/shorte1/Ewan\'s Hard Drive/phd/data/caine_WRF_data/'

In [15]:
base_dir = '/g/data/hj10/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/'
save_dir = '/home/563/esh563/TINT_tracks/'
fig_dir = '/home/563/esh563/TINT_figures/'
ERA5_dir = '/g/data/rt52/era5/pressure-levels/reanalysis/'

In [4]:
dates = np.arange(
    np.datetime64('2005-11-01 00:00'), 
    np.datetime64('2006-04-01 00:00'),
    np.timedelta64(10, 'm'))

filenames, start_time, end_time = cf.CPOL_files_from_datetime_list(
    dates, base_dir=base_dir)

micro_physics = 'cpol'

tracks_obj  = tint.Tracks(params={
    'AMBIENT': 'ERA5', 'AMBIENT_BASE_DIR': ERA5_dir})

Gathering files.


In [5]:
micro_physics = 'lin'

dates = np.arange(
    np.datetime64('2006-02-08 12:00'), 
    np.datetime64('2006-02-13 10:10'),
    np.timedelta64(10, 'm'))

filenames, start_time, end_time = caf.caine_files_from_datetime_list(
    dates, micro_physics=micro_physics, 
    base_dir=WRF_dir)

tracks_obj  = tint.Tracks(params={
    'AMBIENT': 'WRF', 'AMBIENT_BASE_DIR': WRF_dir + 'lind02/'})

Gathering files.


In [4]:
micro_physics = 'thompson'

dates = np.arange(
    np.datetime64('2006-02-08 12:00'), 
    np.datetime64('2006-02-13 10:10'),
    np.timedelta64(10, 'm'))

filenames, start_time, end_time = caf.caine_files_from_datetime_list(
    dates, micro_physics=micro_physics, 
    base_dir=WRF_dir)

tracks_obj  = tint.Tracks(params={
    'AMBIENT': 'WRF', 'AMBIENT_BASE_DIR': WRF_dir + 'thompsond02/'})

Gathering files.


# Create Tracks

In [5]:
b_path = '/home/student.unimelb.edu.au/shorte1/Documents/'
b_path += 'CPOL_analysis/circ_b_ind_set.pkl'
with open(b_path, 'rb') as f:
    b_ind_set = pickle.load(f)

In [6]:
def gen_tracks(tracks_obj, filenames, b_path=None):

    grids = (
        pyart.io.read_grid(fn, include_fields = ['reflectivity'])
        for fn in filenames)
    
      
    tracks_obj.get_tracks(grids, b_path=b_path)
        
    return tracks_obj

In [7]:
tracks_obj = gen_tracks(tracks_obj, filenames, b_path=b_path)

Getting ERA5 metadata.
Getting Intepolated ERA5 for next hour.


/home/student.unimelb.edu.au/shorte1/anaconda3/envs/CPOL_analysis/lib/python3.8/site-packages/numba/core/consts.py:114: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return getattr(value, expr.attr)
/home/student.unimelb.edu.au/shorte1/anaconda3/envs/CPOL_analysis/lib/python3.8/site-packages/numba/core/ir_utils.py:2097: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int

No objects found in scan 0.
No objects found in scan 1.
No objects found in scan 2.
No objects found in scan 3.
No objects found in scan 4.
No objects found in scan 5.
No objects found in scan 6.
No objects found in scan 7.
No objects found in scan 8.
No objects found in scan 9.
No objects found in scan 10.
No objects found in scan 11.
No objects found in scan 12.
No objects found in scan 13.
No objects found in scan 14.
No objects found in scan 15.
No objects found in scan 16.
No objects found in scan 17.
No objects found in scan 18.
No objects found in scan 19.
No objects found in scan 20.
No objects found in scan 21.
No objects found in scan 22.
No objects found in scan 23.
No objects found in scan 24.
No objects found in scan 25.
No objects found in scan 26.
No objects found in scan 27.
No objects found in scan 28.
No objects found in scan 29.
No objects found in scan 30.
No objects found in scan 31.
No objects found in scan 32.
No objects found in scan 33.
No objects found in scan

No objects found in scan 256.
No objects found in scan 257.
No objects found in scan 258.
No objects found in scan 259.
No objects found in scan 260.
No objects found in scan 261.
No objects found in scan 262.
No objects found in scan 263.
No objects found in scan 264.
No objects found in scan 265.
No objects found in scan 266.
No objects found in scan 267.
No objects found in scan 268.
No objects found in scan 269.
No objects found in scan 270.
No objects found in scan 271.
No objects found in scan 272.
No objects found in scan 273.
No objects found in scan 274.
No objects found in scan 275.
No objects found in scan 276.
No objects found in scan 277.
No objects found in scan 278.
No objects found in scan 279.
No objects found in scan 280.
No objects found in scan 281.
No objects found in scan 282.
No objects found in scan 283.
No objects found in scan 284.
No objects found in scan 285.
No objects found in scan 286.
No objects found in scan 287.
No objects found in scan 288.
No objects

No objects found in scan 500.
No objects found in scan 501.
No objects found in scan 502.
No objects found in scan 503.
No objects found in scan 504.
No objects found in scan 505.
No objects found in scan 506.
No objects found in scan 507.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 508.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 509.
Writing tracks for scan 510.
Writing tracks for scan 511.
Writing tracks for scan 512.
Writing tracks for scan 513.
Writing tracks for scan 514.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 515.
Writing tracks for scan 516.
Writing tracks for scan 517.
Writing tracks for scan 518.
Writing tracks for scan 519.
Writing tracks for scan 520.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 521.
Writing tracks for scan 522.
Writing tracks for scan 523.
Writing tracks for scan 524.
Writing tracks for scan 525.
Writing tracks for scan 526.
Getting Intepolated ERA5 for the 

...

No objects found in scan 19269.
No objects found in scan 19270.
Writing tracks for scan 19271.
Writing tracks for scan 19272.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19273.
Writing tracks for scan 19274.
Writing tracks for scan 19275.
Writing tracks for scan 19276.
No objects found in scan 19277.
No objects found in scan 19278.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19279.
Writing tracks for scan 19280.
No objects found in scan 19281.
Writing tracks for scan 19282.
Writing tracks for scan 19283.
Writing tracks for scan 19284.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19285.
Writing tracks for scan 19286.
Writing tracks for scan 19287.
No objects found in scan 19288.
No objects found in scan 19289.
No objects found in scan 19290.
No objects found in scan 19291.
No objects found in scan 19292.
No objects found in scan 19293.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19294.
No obje

No objects found in scan 19506.
No objects found in scan 19507.
No objects found in scan 19508.
No objects found in scan 19509.
No objects found in scan 19510.
No objects found in scan 19511.
No objects found in scan 19512.
No objects found in scan 19513.
No objects found in scan 19514.
No objects found in scan 19515.
No objects found in scan 19516.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19517.
Writing tracks for scan 19518.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19519.
Writing tracks for scan 19520.
Writing tracks for scan 19521.
Writing tracks for scan 19522.
Writing tracks for scan 19523.
Writing tracks for scan 19524.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19525.
Writing tracks for scan 19526.
Writing tracks for scan 19527.
Writing tracks for scan 19528.
Writing tracks for scan 19529.
Writing tracks for scan 19530.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19531.
Writing

No objects found in scan 19721.
No objects found in scan 19722.
No objects found in scan 19723.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19724.
Writing tracks for scan 19725.
Writing tracks for scan 19726.
Writing tracks for scan 19727.
Writing tracks for scan 19728.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19729.
Writing tracks for scan 19730.
Writing tracks for scan 19731.
Writing tracks for scan 19732.
Writing tracks for scan 19733.
Writing tracks for scan 19734.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19735.
Writing tracks for scan 19736.
Writing tracks for scan 19737.
Writing tracks for scan 19738.
Writing tracks for scan 19739.
Writing tracks for scan 19740.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19741.
Writing tracks for scan 19742.
Writing tracks for scan 19743.
Writing tracks for scan 19744.
Writing tracks for scan 19745.
Writing tracks for scan 19746.
Getting Intepol

Writing tracks for scan 19934.
Writing tracks for scan 19935.
Writing tracks for scan 19936.
Writing tracks for scan 19937.
Writing tracks for scan 19938.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19939.
Writing tracks for scan 19940.
Writing tracks for scan 19941.
Writing tracks for scan 19942.
Writing tracks for scan 19943.
Writing tracks for scan 19944.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 19945.
Writing tracks for scan 19946.
Writing tracks for scan 19947.
Writing tracks for scan 19948.
Writing tracks for scan 19949.
Writing tracks for scan 19950.
No objects found in scan 19951.
No objects found in scan 19952.
No objects found in scan 19953.
No objects found in scan 19954.
No objects found in scan 19955.
No objects found in scan 19956.
No objects found in scan 19957.
No objects found in scan 19958.
No objects found in scan 19959.
No objects found in scan 19960.
No objects found in scan 19961.
No objects found in scan 19962.
N

Writing tracks for scan 20157.
Writing tracks for scan 20158.
Writing tracks for scan 20159.
Writing tracks for scan 20160.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 20161.
Writing tracks for scan 20162.
Writing tracks for scan 20163.
Writing tracks for scan 20164.
Writing tracks for scan 20165.
Writing tracks for scan 20166.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 20167.
Writing tracks for scan 20168.
Writing tracks for scan 20169.
Writing tracks for scan 20170.
Writing tracks for scan 20171.
Writing tracks for scan 20172.
No objects found in scan 20173.
No objects found in scan 20174.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 20175.
Writing tracks for scan 20176.
Writing tracks for scan 20177.
Writing tracks for scan 20178.
Getting Intepolated ERA5 for the next hour.
Writing tracks for scan 20179.
Writing tracks for scan 20180.
Writing tracks for scan 20181.
Writing tracks for scan 20182.
Writing tracks f

In [8]:
current_time = str(datetime.datetime.now())[0:-7]
current_time = current_time.replace(" ", "_").replace(":", "_")
current_time = current_time.replace("-", "")
out_file_name = save_dir + 'cpol_2005_2006.pkl'.format(
    micro_physics, current_time)
with open(out_file_name, 'wb') as f:
    pickle.dump(tracks_obj, f)

In [9]:
tracks_obj.params

{'SEARCH_MARGIN': 50000,
 'FLOW_MARGIN': 40000,
 'MAX_FLOW_MAG': 60,
 'MAX_DISPARITY': 999,
 'MAX_SHIFT_DISP': 60,
 'ISO_SMOOTH': 3,
 'GS_ALT': 2000,
 'LEVELS': array([[  500,  3500],
        [ 3500,  7500],
        [ 7500, 10000]]),
 'MIN_SIZE': [80, 400, 800],
 'FIELD_THRESH': ['convective', 20, 15],
 'ISO_THRESH': [10, 10, 10],
 'TRACK_INTERVAL': 0,
 'CELL_THRESH': 25,
 'CELL_START': 3000,
 'RAIN': False,
 'SAVE_RAIN': False,
 'AMBIENT': 'ERA5',
 'AMBIENT_BASE_DIR': "/media/shorte1/Ewan's Hard Drive/phd/data/era5/pressure-levels/reanalysis/",
 'DT': 10,
 'CLASS_THRESH': {'OFFSET_MAG': 10000,
  'SHEAR_MAG': 2,
  'VEL_MAG': 2,
  'REL_VEL_MAG': 2,
  'ANGLE_BUFFER': 10},
 'EXCL_THRESH': {'SMALL_AREA': 500,
  'LARGE_AREA': 50000,
  'BORD_THRESH': 0.005,
  'MAJOR_AXIS_LENGTH': 100,
  'AXIS_RATIO': 3,
  'DURATION': 30},
 'BOUNDARY_GRID_CELLS': {(0, 48),
  (0, 49),
  (0, 50),
  (0, 51),
  (0, 52),
  (0, 53),
  (0, 54),
  (0, 55),
  (0, 56),
  (0, 57),
  (0, 58),
  (0, 59),
  (0, 60),
  (0, 

In [12]:
tracks_obj = tint.objects.classify_tracks(tracks_obj)
tracks_obj = tint.objects.get_exclusion_categories(tracks_obj)

In [8]:
with open(save_dir + 'cpol_2005_2006.pkl', 'rb') as f:
    tracks_obj = pickle.load(f)

In [9]:
tracks_obj.params

{'SEARCH_MARGIN': 50000,
 'FLOW_MARGIN': 40000,
 'MAX_FLOW_MAG': 60,
 'MAX_DISPARITY': 999,
 'MAX_SHIFT_DISP': 60,
 'ISO_SMOOTH': 3,
 'GS_ALT': 1500,
 'LEVELS': array([[  500,  3500],
        [ 3500,  7500],
        [ 7500, 10000]]),
 'MIN_SIZE': [80, 400, 800],
 'FIELD_THRESH': ['convective', 20, 15],
 'ISO_THRESH': [10, 10, 10],
 'TRACK_INTERVAL': 0,
 'CELL_THRESH': 25,
 'CELL_START': 3000,
 'RAIN': False,
 'SAVE_RAIN': False,
 'AMBIENT': 'ERA5',
 'AMBIENT_BASE_DIR': "/media/shorte1/Ewan's Hard Drive/phd/data/era5/pressure-levels/reanalysis/",
 'CLASS_THRESH': {'OFFSET_MAG': 5000,
  'SHEAR_MAG': 1,
  'VEL_MAG': 1,
  'REL_VEL_MAG': 1,
  'ANGLE_BUFFER': 0.17453292519943295},
 'EXCL_THRESH': {'SMALL_AREA': 500,
  'LARGE_AREA': 50000,
  'BORD_THRESH': 0.01,
  'MAJOR_AXIS_LENGTH': 100,
  'AXIS_RATIO': 3},
 'BOUNDARY_GRID_CELLS': {(0, 48),
  (0, 49),
  (0, 50),
  (0, 51),
  (0, 52),
  (0, 53),
  (0, 54),
  (0, 55),
  (0, 56),
  (0, 57),
  (0, 58),
  (0, 59),
  (0, 60),
  (0, 61),
  (0, 62)

In [11]:
def fill_end(group_df):
    import pdb; pdb.set_trace()
    group_df[['u_shift', 'v_shift']].iloc[-1] = group_df[['u_shift', 'v_shift']].iloc[-2]
    

In [12]:
tracks_obj.tracks.groupby(level=['uid', 'level']).apply(
        lambda x: fill_end(x))

> /tmp/ipykernel_31696/4190406837.py(3)fill_end()
      1 def fill_end(group_df):
      2     import pdb; pdb.set_trace()
----> 3     group_df[['u_shift', 'v_shift']].iloc[-1] = group_df[['u_shift', 'v_shift']].iloc[-2]
      4 

ipdb> group_df['u_shift'].iloc[-1]
-4.16
ipdb> group_df['u_shift'].iloc[-1] = 5
ipdb> group_df['u_shift'].iloc[-1]
5.0
ipdb> q


BdbQuit: 

> /home/student.unimelb.edu.au/shorte1/anaconda3/envs/CPOL_analysis/lib/python3.8/bdb.py(113)dispatch_line()
    111         if self.stop_here(frame) or self.break_here(frame):
    112             self.user_line(frame)
--> 113             if self.quitting: raise BdbQuit
    114         return self.trace_dispatch
    115 

ipdb> q


In [11]:
tracks_obj.tracks_class['offset_type']

scan  time                 level  uid
26    2006-02-13 04:20:00  0      0      Parallel 
                           1      0      Parallel 
                           2      0      Parallel 
27    2006-02-13 04:30:00  0      0      Parallel 
                           1      0      Parallel 
                                           ...    
109   2006-02-13 18:20:00  1      6      Trailing 
                           2      6      Trailing 
110   2006-02-13 18:30:00  0      6      Trailing 
                           1      6      Trailing 
                           2      6      Trailing 
Name: offset_type, Length: 186, dtype: object

In [15]:
tracks_obj.tracks_class

grid_x         grid_y        lon  \
scan time                level uid                                           
26   2006-02-13 04:20:00 0     0    94258.747101   91626.815796  131.90924   
                         1     0    93731.021881   89518.089294  131.90446   
                         2     0    94601.573944   90367.965698  131.91242   
27   2006-02-13 04:30:00 0     0    94103.031158   93521.881104  131.90776   
                         1     0    91883.563995   89156.665802  131.88752   
...                                          ...            ...        ...   
109  2006-02-13 18:20:00 1     6   -42433.776855 -111225.147247  130.65241   
                         2     6   -39183.788300 -115186.266899  130.68239   
110  2006-02-13 18:30:00 0     6   -59737.138748 -104407.777786  130.49269   
                         1     6   -47207.069397 -108781.957626  130.60835   
                         2     6   -44592.847824 -111780.140400  130.63246   

                                         lat inflow_type  \
scan time                level uid                         
26   2006-02-13 04:20:00 0     0   -11.42382    Rear Fed   
                         1     0   -11.44280    Rear Fed   
                         2     0   -11.43513   Ambiguous   
27   2006-02-13 04:30:00 0     0   -11.40678    Rear Fed   
                         1     0   -11.44610    Rear Fed   
...                                      ...         ...   
109  2006-02-13 18:20:00 1     6   -13.24915   Front Fed   
                         2     6   -13.28482   Front Fed   
110  2006-02-13 18:30:00 0     6   -13.18757   Front Fed   
                         1     6   -13.22712   Front Fed   
                         2     6   -13.25412   Front Fed   

                                          propagation_type          tilt_type  \
scan time                level uid                                              
26   2006-02-13 04:20:00 0     0      Up-Shear Propagating  Ambiguous (Small    
                         1     0    Down-Shear Propagating  Ambiguous (Small    
                         2     0    Ambiguous (Low Relativ  Ambiguous (Small    
27   2006-02-13 04:30:00 0     0      Up-Shear Propagating  Ambiguous (Small    
                         1     0    Down-Shear Propagating  Ambiguous (Small    
...                                                    ...                ...   
109  2006-02-13 18:20:00 1     6      Up-Shear Propagating  Down-Shear Tilted   
                         2     6      Up-Shear Propagating  Down-Shear Tilted   
110  2006-02-13 18:30:00 0     6    Down-Shear Propagating    Up-Shear Tilted   
                         1     6      Up-Shear Propagating  Down-Shear Tilted   
                         2     6      Up-Shear Propagating  Down-Shear Tilted   

                                   offset_type  
scan time                level uid              
26   2006-02-13 04:20:00 0     0     Parallel   
                         1     0     Parallel   
                         2     0     Parallel   
27   2006-02-13 04:30:00 0     0     Parallel   
                         1     0     Parallel   
...                                        ...  
109  2006-02-13 18:20:00 1     6     Trailing   
                         2     6     Trailing   
110  2006-02-13 18:30:00 0     6     Trailing   
                         1     6     Trailing   
                         2     6     Trailing   

[186 rows x 8 columns]

In [36]:
test = copy.deepcopy(tracks_obj.tracks)
inflow_cond = (
    tracks_obj.system_tracks['x_vert_disp'] * tracks_obj.tracks['u_shear']
    + tracks_obj.system_tracks['y_vert_disp'] * tracks_obj.tracks['v_shear'])

In [41]:
u_shear

scan  time                 level  uid
26    2006-02-13 04:20:00  0      0     -6.00483
                           1      0      2.92783
                           2      0      2.61377
27    2006-02-13 04:30:00  0      0     -6.07322
                           1      0      3.06301
                                          ...   
109   2006-02-13 18:20:00  1      6      3.84559
                           2      6      3.51672
110   2006-02-13 18:30:00  0      6     -8.69906
                           1      6      3.89933
                           2      6      3.51533
Name: u_shear, Length: 186, dtype: float64

In [43]:
x_offset

scan  time                 uid
26    2006-02-13 04:20:00  0        342.826843
27    2006-02-13 04:30:00  0       -160.064697
28    2006-02-13 04:40:00  0       4056.358337
29    2006-02-13 04:50:00  0       5448.055267
30    2006-02-13 05:00:00  0       7804.946899
                                      ...     
106   2006-02-13 17:50:00  6      13907.232285
107   2006-02-13 18:00:00  6      24080.314636
108   2006-02-13 18:10:00  6      11994.714737
109   2006-02-13 18:20:00  6      16506.528854
110   2006-02-13 18:30:00  6      15144.290924
Name: x_vert_disp, Length: 62, dtype: float64

In [40]:
tracks_obj.system_tracks['x_vert_disp'] * tracks_obj.tracks['u_shear']

scan  time                 uid  level
26    2006-02-13 04:20:00  0    0         -2058.616913
                                1          1003.738717
                                2           896.070518
27    2006-02-13 04:30:00  0    0           972.108121
                                1          -490.279768
                                             ...      
109   2006-02-13 18:20:00  6    1         63477.342297
                                2         58048.840153
110   2006-02-13 18:30:00  6    0       -131741.095406
                                1         59052.587929
                                2         53237.180214
Length: 186, dtype: float64